# Elasticsearch Retrieval with Claude

This notebook provides a step-by-step guide for using the Elasticsearch search tool with Claude. We will:

1. Set up the environment and imports
2. Build a search tool to query an Elasticsearch instance
3. Test the search tool  
4. Create a Claude client with access to the tool 
5. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [ ]:
import os
import sys
import dotenv
import anthropic

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv()

In [ ]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

# Store your data

The first step is setting up your datastore. Here, we will make use of the [Kaggle Amazon Products 2020 Dataset](https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020). It contains 10000 products from Amazon, including their product title, description, price, category tags, etc. For the purposes of this notebook, we've pre-processed the data to concatenate the title, description and category tags into a single "document" field and saved it locally as a JSONL with one line for each product.

We now need to transform this raw text dataset into an embedding dataset. In this notebook we will opt for the simplest possible way to do this locally:

1. We will use the [sentence-transformers](https://www.sbert.net/index.html) library, which allows us to use a lightweight model to embed our text data using only a CPU if that is all we have available.
2. We will save the text/embedding pairs on disk as a JSONL file that can be loaded in memory on the fly.

Local methods like this work quite well for small datasets, but for larger datasets you may want to consider using a cloud-based method to both create the embeddings and store the vector datastore. These methods are covered in the [Remote Retrieval](remote-retrieval.ipynb) notebook.

In [ ]:
# Set up Elasticsearch and upload the data

from elasticsearch import Elasticsearch

cloud_id = os.getenv("ELASTICSEARCH_CLOUD_ID")
api_key_id = os.getenv("ELASTICSEARCH_API_KEY_ID")
api_key = os.getenv("ELASTICSEARCH_API_KEY")

index_name = "amazon-products-database"

if cloud_id is None or api_key_id is None or api_key is None:
    raise ValueError("ELASTICSEARCH_CLOUD_ID, ELASTICSEARCH_API_KEY_ID, and ELASTICSEARCH_API_KEY must be set as environment variables")

es = Elasticsearch(
        cloud_id=cloud_id,
        api_key=(api_key_id, api_key),
    )
    
if not es.indices.exists(index=index_name):
    from claude_retriever.utils import upload_to_elasticsearch
    upload_to_elasticsearch(
        input_file="data/amazon-products.jsonl",
        index_name=index_name,
        cloud_id=cloud_id,
        api_key_id=api_key_id,
        api_key=api_key
    )

# Create a Search Tool for your data

We now create a Search Tool, which can take queries and return formatted relevant results. We also need to describe what the search tool will return, which Claude will read to make sure it is correctly used.

In [ ]:
from claude_retriever.searcher.searchtools.elasticsearch import ElasticsearchCloudSearchTool

AMAZON_SEARCH_TOOL_DESCRIPTION = 'The search engine will search over the Amazon Product database, and return for each product its title, description, and a set of tags.'
amazon_search_tool = ElasticsearchCloudSearchTool(tool_description=AMAZON_SEARCH_TOOL_DESCRIPTION,
                                                  elasticsearch_cloud_id=cloud_id,
                                                  elasticsearch_api_key_id=api_key_id,
                                                  elasticsearch_api_key=api_key,
                                                  elasticsearch_index=index_name)

Let's test it to see if the tool works!

In [ ]:
dinos = amazon_search_tool.search("fun kids dinosaur book", n_search_results_to_use=3)
print(dinos)

# Use Claude with Retrieval

We can now simply pass this search tool to Claude to use, much in the same way a person might.

In [ ]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], search_tool = amazon_search_tool)

query = "I want to get my daughter more interested in science. What kind of gifts should I get her?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (no access to the tool).

In [ ]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)

Now we get the same completion, but give Claude the ability to use the tool when thinking about the response.

In [ ]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)

Often, you'll want finer-grained control about how exactly Claude uses the results. For this workflow we recommend "retrieve then complete".

In [ ]:
relevant_search_results = client.retrieve(
    query=query,
    stop_sequences=[anthropic.HUMAN_PROMPT, 'END_OF_SEARCH'],
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_searches_to_try=5,
    max_tokens_to_sample=1000)

print('-'*50)
print('Relevant results:')
print(relevant_search_results)
print('-'*50)

In [ ]:
qa_prompt = f'''{anthropic.HUMAN_PROMPT} You are a friendly product recommender. Here is a query issued by a user looking for product recommendations:

{query}

Here are a set of search results that might be helpful for answering the user's query:

{relevant_search_results}

Once again, here is the user's query:

<query>{query}</query>

Please write a response to the user that answers their query and provides them with helpful product recommendations. Feel free to use the search results above to help you write your response, or ignore them if they are not helpful.

At the end of your response, under "Products you might like:", list the top 3 product names from the search results that you think the user would most like.

Please ensure your results are in the following format:

<result>
Your response to the user's query.
</result>
<recommendations>
Products you might like:
1. Product name
2. Product name
3. Product name
</recommendations>{anthropic.AI_PROMPT}'''

response = client.completions.create(
    prompt=qa_prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)

print('-'*50)
print('Response:')
print(response.completion)
print('-'*50)